In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
path_train = "data/fhv_tripdata_2021-01.parquet"
path_val = "data/fhv_tripdata_2021-02.parquet"

# Q1 - Read the data for January. How many records are there?

- 1054112
- 1154112
- 1254112
- 1354112

In [3]:
df = pd.read_parquet(path_train)
df.shape

(1154112, 7)

# Q2 - What's the average trip duration in January?
- 15.16
- 19.16
- 24.16
- 29.16

In [4]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [10]:
df['duration'] = df.dropOff_datetime -df.pickup_datetime
df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
df['duration'].mean()

19.1672240937939

In [12]:
print("Amount of dropped columns =", 1154112-1109826)

Amount of dropped columns = 44286


In [11]:
df2 = df[(df.duration >=1) & (df.duration<=60)]
df2.shape

(1109826, 8)

# Q3. What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.
- 53%
- 63%
- 73%
- 83%

In [15]:
total = df2.shape[0]
total_missing = df2.PUlocationID.isnull().sum()

ratio = (total_missing/total)*100

print(f"Fraction of missing values for PUlocation ID is : {ratio}%")

Fraction of missing values for PUlocation ID is : 83.52732770722618%


In [18]:
df2.PUlocationID.fillna(-1, inplace=True)
df2.DOlocationID.fillna(-1, inplace=True)

c:\Users\micha\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


# Q4 One-hot encoding
### Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it
#### What's the dimensionality of this matrix? (The number of columns).

- 2
- 152
- 352
- 525
- 725

In [21]:
categorical = ['PUlocationID', 'DOlocationID']
df2[categorical] = df2[categorical].astype(str)
train_dicts = df2[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
type(X_train)

c:\Users\micha\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


scipy.sparse.csr.csr_matrix

In [22]:
X_train.shape

(1109826, 525)

# Q5 - Training a model
### Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data
### What's the RMSE on train?

- 5.52
- 10.52
- 15.52
- 20.52

In [23]:
target = 'duration'
y_train = df2[target].values



In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107202131

# Q6 Evaluating the model

- Now let's apply this model to the validation dataset (Feb 2021).

### What's the RMSE on validation?

- 6.01
- 11.01
- 16.01
- 21.01

In [26]:
def read_data(file_path):
    df = pd.read_parquet(file_path)

    df['duration'] = df.dropOff_datetime -df.pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >=1) & (df.duration<=60)]

    df.PUlocationID.fillna(-1, inplace=True)
    df.DOlocationID.fillna(-1, inplace=True)

    return df
    

In [29]:
df_val = read_data(path_val)

categorical = ['PUlocationID', 'DOlocationID']

df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

In [30]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.01428311525346